In [1]:
import numpy as np
import pandas as pd

## To Explain

### Decision Tree
* purity/impurity
* entropy vs. Gini index
* using decision tree for prediction

### Random Forest
* bootstrap aggregating
* OOB error estimating
* pros and cons of random forest

### Extra
* compare classification with sci-kit learn functions vs. random forest

In [2]:
data = pd.read_csv("ChurnTest.csv")
data.head()

,X1,X2,Y
0,619,0,1
1,502,0,1
2,645,1,1
3,822,1,0
4,376,0,1


In [18]:
# function to calculate metric for split (entropy)
def calc_entropy(obs_list):
    ent = 0
    for unique_class in set(obs_list.iloc[:,-1]):
        prop = len(obs_list[obs_list.iloc[:,-1]==unique_class]) / len(obs_list)
        ent += (-1 * prop) * np.log2(prop)
    return ent

calc_entropy(data)

unique_class:  1
prop:  1.0


0.0

In [6]:
# function to build decision tree
def build_tree(params):
    return "tree has been built!"

In [7]:
# function to split data

In [ ]:
# load data

In [2]:
# set number of trees

In [3]:
# set number of splits

In [4]:
# bootstrap datasets

In [8]:
# build decision tree

In [1]:
# main function

#### References
http://www.cs.cmu.edu/afs/cs.cmu.edu/academic/class/15381-s06/www/DTs.pdf